Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

NAME = "Oscar Delgado"
COLLABORATORS = "Colin Murphy"

---

In [ ]:
import $file.hw4stdlib
import hw4stdlib._

# Homework 4

This homework is a bit longer because of the Exam on Thursday. Thus it won't be due until June 25th at 11:59pm

In this assignment we will develop our first interpreter. But before this we will need to define the Integers and the arithmetic operations for them.

# $\mathbb{Z}$: The Integers

For the arithmetic interpreter we will write later on in the homework we will need arithmetic operations for the integers. These will obviously correspond very closely to the operations defined on the Natural Numbers. But now we will need to account for negative numbers in a proper way. 

Start by defining the Algebraic Datatype(sealed trait with case classes) for the Integers. The encoded type should be:

$$
\begin{align}
    \mathbb{Z} ::=&\ \text{Positive}\ \mathbb{N}\\
    \mid&\ \text{Negative}\ \mathbb{N}
\end{align}
$$

We will use this encoding because it will allow us to use some of our definitions for the Natural Numbers to define the operations on the Integers. This is the core strategy of functional programming: using smaller functions to build up larger ones. This is true even across types. We want to always minimize the ammount of code we have be reusing what we have already constructed, whenever possible. 

## Defining $\mathbb{Z}$ (5 points)

Define the ADT for Integers below: 

In [ ]:
sealed trait Integer
case class Positive(Pred : Nat) extends Integer
case class Negative(Pred : Nat) extends Integer

If your definition was correct the following functions should define and print some integers



In [ ]:
//Autograder cell -- do not edit, delete, or duplicate.
def pos_one : Integer = Positive(Succ(Zero))
def neg_one : Integer = Negative(Succ(Zero))

def int_to_str(x : Integer) : String = x match {
    case Positive(x) => nat_to_str(x)
    case Negative(x) => "-" + nat_to_str(x)
}

def print_integer(x : Integer) = println(int_to_str(x))
assert(int_to_str(pos_one) equals "1")
assert(int_to_str(neg_one) equals "-1")
print_integer(pos_one)
print_integer(neg_one)

## Absolute Value and Negation 

### Absolute Value (5 points)

Define the absolute value function for Integers. This should be a very simple function that takes an Integer and gives its absolute value as a $\mathbb{N}$

In [ ]:
def abs(x : Integer) : Nat = x match{
    case Positive(x) => x
    case Negative(x) => x
}

In [ ]:
//Autograder cell -- do not edit, delete, or duplicate.
assert(abs(Positive(two)) equals two)
assert(abs(Negative(two)) equals two)
assert(abs(Negative(Zero)) equals abs(Positive(Zero)))


### Negation (5 points)

This should be very easy as well. Define the negation function. This makes a positive Integer negative and a negative Integer positive.

In [ ]:
def negate(x : Integer) : Integer = x match{
    case Positive(x) => Negative(x)
    case Negative(x) => Positive(x)
}

In [ ]:
//Autograder cell -- do not edit, delete, or duplicate.
assert(negate(Positive(five)) equals Negative(five))
assert(negate(Negative(seven)) equals Positive(seven))
assert(negate(negate(Positive(five))) equals Positive(five))

## Addition and Subtraction

Define plus and minus for Integers. Don't use the versions we created for $\mathbb{N}$ as it did some funky things to get minus to behave correctly. Try and create these from scratch instead. It is recommended to use the lte(less than or equals) function we defined last week along with abs to make the job easier. 

Note that we have renamed the operations for natural numbers so that they are of the form:



``` nat_<operation name> ```

For instance, plus has been renamed to ``` nat_plus ```. This was done so we don't have namespace conflicts

### Addition (5 points)

In [ ]:
def plus(n : Integer, m : Integer) : Integer = (n,m) match{
    case (n,Positive(Zero)) => n
    case (n,Negative(Zero)) => n
    case (Positive(Zero),m) => m
    case (Negative(Zero),m) => m
    case (Positive(Succ(n)),Positive(m)) => plus(Positive(n),Positive(Succ(m))) 
    case (Negative(Succ(n)),Negative(m)) => plus(Negative(n),Negative(Succ(m)))
    case (Positive(Succ(n)),Negative(Succ(m)))=> nat_lte(Succ(n),Succ(m)) match {
        case False => plus(Positive(n),Negative(m))
        case True => plus(Negative(m),Positive(n))
    }
    case (Negative(n),Positive(m)) =>plus(Positive(m),Negative(n))
    
}


In [ ]:
//Autograder cell -- do not edit, delete, or duplicate.
assert(plus(pos_one,pos_one) equals Positive(two))
assert((plus(pos_one,neg_one) equals Positive(Zero)) ||
       (plus(pos_one,neg_one) equals Negative(Zero)))
assert(plus(Positive(five),Negative(two)) equals Positive(three))
assert(plus(Positive(five),Positive(two)) equals Positive(seven))
assert(plus(Positive(five),Negative(seven)) equals Negative(two))

### Subtraction (5 points)

Hint: Subtraction is very easy if you use ```plus``` and ```negate```

In [ ]:
def minus(x : Integer, y : Integer) : Integer = (x,y) match{
    case (x,y) => plus(x,negate(y))
}

In [ ]:
//Autograder cell -- do not edit, delete, or duplicate.
assert(minus(pos_one,neg_one) equals Positive(two))
assert((minus(pos_one,pos_one) equals Positive(Zero)) ||
       (minus(pos_one,pos_one) equals Negative(Zero)))
assert(minus(Positive(five),Positive(two)) equals Positive(three))
assert(minus(Negative(two),Positive(three)) equals Negative(five))
assert(minus(Positive(five),Negative(two)) equals Positive(seven))
assert(minus(Negative(three),Negative(seven)) equals Positive(four))

## Multiplication and Exponentiation

Write multiplication and exponentiation for Integers. You should be able to use ```nat_mult``` to greatly simplify this

### Multiplication (5 points)

In [ ]:
def mult_helper(x : Nat, acc : Nat, y : Nat) : Nat = (x,acc,y) match{
    case (x,acc,Zero) => acc
    case (x,acc,Succ(y)) =>mult_helper(x,nat_plus(acc,x),y)
}

def mult(x : Integer, y : Integer) : Integer = (x,y) match{
    case (x,Positive(Zero)) => Positive(Zero)
    case (x,Negative(Zero)) => Positive(Zero)
    case (Positive(Zero),y) => Positive(Zero)
    case (Negative(Zero),y) => Positive(Zero)
    case (Positive(x),Positive(y)) => Positive(mult_helper(x,Zero,y))
    case (Negative(x),Negative(y)) => Positive(mult_helper(x,Zero,y))
    case _=> Negative(mult_helper(abs(x),Zero,abs(y)))
    
}

In [ ]:
//Autograder cell -- do not edit, delete, or duplicate.
assert(mult(Positive(two),Positive(three)) equals Positive(six))
assert(mult(Positive(two),Negative(three)) equals Negative(six))
assert(mult(Negative(one),Positive(five)) equals Negative(five))
assert(mult(Negative(one),Negative(four)) equals Positive(four))

### Exponentiation (5 points)

recall that for ``pow`` we will restrict ourselves to only positive powers. Use the definition of `mult` from above so that your polarity(Positive/Negative) is correct. Recall the cases for $-x^n$ for even vs odd $n$. A hint for your base case: `pow(x, 0) = Positive( 1 )`

In [ ]:
def pow(x : Integer, y : Nat) : Integer = (x,y) match{
    case (x,Zero) => Positive(one)
    case (x,Succ(y)) => mult(x,pow(x,y))
}

In [ ]:
//Autograder cell -- do not edit, delete, or duplicate.
assert(pow(Positive(two),three) equals Positive(eight))
assert(pow(Negative(two),three) equals Negative(eight))
assert(pow(Negative(one),Zero) equals Positive(one))

# The Arithmetic Language: Our First Interpreter

Now we are ready to define our first interpreter. We will define the Arithmetic language syntax below as a sealed trait. It will be your job to correctly construct the interpreter for it based on the inference rules we covered in class. Recall that each rule corresponds to a case in the `eval` function.


In [ ]:
sealed trait Expr 
case class Num(x : Integer) extends Expr 
case class Plus(x : Expr, y : Expr) extends Expr
case class Minus(x : Expr, y : Expr) extends Expr
case class Mult(x : Expr, y : Expr) extends Expr
case class Pow(x : Expr, y : Nat) extends Expr

## The Interpreter (15 points)

Now that we have defined the syntax for Arithmetic expressions, go ahead and define the interpreter. We have given the signature for the function. (Bonus points if you define and use the helper function `eval-bin` that we discussed in class.

Recall that the type of this interpreter should be $eval : \text{Expr} \rightarrow \mathbb{Z}$

In [ ]:
def eval_bin (f : ((Integer, Integer) => Integer), e1 : Expr, e2 : Expr) : Integer = (e1,e2) match{
    case (Num(e1),Num(e2)) => f(e1,e2)
}

In [ ]:
def eval(expr : Expr) : Integer = expr match{
    case Num(x) => x
    case Plus(x,y) => eval_bin(plus,Num(eval(x)),Num(eval(y)))
    case Minus(x,y) => eval_bin(minus,Num(eval(x)),Num(eval(y)))
    case Mult(x,y) => eval_bin(mult,Num(eval(x)),Num(eval(y)))
    case Pow(x,y) => pow(eval(x),y)
}

In [ ]:
//Autograder cell -- do not edit, delete, or duplicate.
val expr1:Expr = Plus(Num(Positive(five)),Num(Negative(three)))//2
val expr2:Expr = Minus(Num(Positive(three)),Num(Negative(two)))//5
val expr3:Expr = Mult(expr1,expr2)//10
val expr4:Expr = Minus(expr3,Num(Positive(eight)))//2
val expr5:Expr = Pow(expr3,two)//100
val expr6:Expr = Mult(expr3,expr3)//100
assert(eval(Num(pos_one)) equals pos_one)
assert(eval(expr1) equals Positive(two))
assert(eval(expr2) equals Positive(five))
assert(eval(expr3) equals Positive(ten))
assert(eval(expr4) equals Positive(two))
assert(eval(expr5) equals eval(expr6))